In [1]:
!pip install jsonlines
import jsonlines
!pip install --no-cache-dir transformers sentencepiece
!pip install langdetect
# !pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]
# from txtai.pipeline import Translation
from langdetect import detect
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import json
import glob
import nltk
nltk.download('punkt')
from copy import deepcopy
from collections import defaultdict
from tqdm import tqdm
from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

     |████████████████████████████████| 4.2 MB 30.4 MB/s 
     |████████████████████████████████| 1.2 MB 33.8 MB/s 
     |████████████████████████████████| 6.6 MB 61.6 MB/s 
     |████████████████████████████████| 84 kB 80.0 MB/s 
     |████████████████████████████████| 596 kB 74.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 981 kB 30.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=2aacae6546d5ab61ec672a76f419a658e225b7b92ef01376a62d5c7f6d0911e8
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [29]:
# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_results.json") as f:
# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_ans_results_ctxlang.json") as f:
# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_ans_across_lang_nofilter.json") as f:
# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_results_100_long.json") as f:
with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_results_100_long_ctxlang.json") as f:
  results = json.load(f)

In [30]:
len(results)

5599

In [9]:
for r in results[:1]:
  print(r.keys())
  print(r)
  # print(r["question"])
  for c in r["ctxs"]:
    # print(c.keys())
    print(c)
  # break

dict_keys(['q_id', 'question', 'answers', 'lang', 'ctxs'])
{'q_id': '-1018987027025691298', 'question': 'ఒరెగాన్ రాష్ట్రంలోని అతిపెద్ద నగరం ఏది ?', 'answers': ['పోర్ట్\u200cల్యాండ్'], 'lang': 'te', 'ctxs': [{'id': '30837326', 'title': 'పోర్ట్ లాండ్ , ఒరెగాన్', 'text': 'మాపింగ్ రీసర్చ్ లో ఉన్న అర్బన్ గ్రీన్ స్పేస్ ఇన్స్టిట్యూట్, నిర్మించబడిన మరియు సహజసిద్ధమైన పర్యావరణాల యొక్క మెరుగైన సమన్వయాన్ని ప్రోత్సహిస్తుంది. ఈ సంస్థ స్థానిక మరియు ప్రాంతీయ రెండు స్థాయిలలో పట్టణ ఉద్యానవనం, కాలిబాట, మరియు సహజసిద్ధమైన ప్రదేశాల ప్రణాలికా సమస్యల పైన పనిచేస్తుంది. "గ్రిస్ట్" పత్రిక ప్రకారం, పోర్ట్ ల్యాండ్ ఐస్ ల్యాండ్ లోని రెక్జవిక్ తర్వాత ప్రపంచంలోనే రెండవ అత్యంత పర్యావరణానికి అనుకూలమైన లేక "పచ్చని" నగరం. 2010లో మూవ్.కామ్ తన "మొదటి పది పచ్చని నగరాలు" జాబితాలో పోర్ట్ లాండును ఉంచింది. ఒరెగాన్ అత్యున్నత నాయస్థానంచే సమర్థించబడుతున్న ఒరెగాన్ రాజ్యాంగం యొక్క బలమైన మాట్లాడే స్వేచ్ఛ పరిరక్షణ వలన, హెన్రీ వి.ఒరెగాన్ రాజ్యాంగం 1987 ప్రకారం ముఖ్యంగా పూర్తి నగ్నత్వం మరియు స్ట్రిప్ (బట్టలు విప్పుట)క్లబ్బులలో లాప్ నృత్య

In [26]:
# all_code = {'ar', 'bn', 'ja', 'fi', 'ru', 'ko', 'te'}
valid = []
for r in results:
  context_languages = set()
  for c in r["ctxs"]:
    if c['ctx_lang'] is not None:
        context_languages.add(c['ctx_lang'])
  # context_languages = context_languages.intersection(all_code)
  if len(context_languages) > 1:
    valid.append(r)
  # else:
  #   print(context_languages)

In [27]:
len(valid)

5001

In [28]:
with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_results_100_long_ctxlang_across_lang_nofilter.json", "w") as outfile:
  json.dump(valid, outfile)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ivanlau/language-detection-fine-tuned-on-xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("ivanlau/language-detection-fine-tuned-on-xlm-roberta-base")

# input = c["text"]
input = "노르딕 국가는 북유럽의 다섯 나라를 말한다. 이들 나라는 북유럽 이사회의 회원국이다. 다음 자치구역은"
input = tokenizer(input, return_tensors='pt')

model = model.to(device)
input = input.to(device)

languages = ['Arabic', 'Basque', 'Breton', 'Catalan', 'Chinese_China', 'Chinese_Hongkong', 
             'Chinese_Taiwan', 'Chuvash', 'Czech', 'Dhivehi', 'Dutch', 'English', 'Esperanto', 
             'Estonian', 'French', 'Frisian', 'Georgian', 'German', 'Greek', 'Hakha_Chin', 
             'Indonesian', 'Interlingua', 'Italian', 'Japanese', 'Kabyle', 'Kinyarwanda', 
             'Kyrgyz', 'Latvian', 'Maltese', 'Mangolian', 'Persian', 'Polish', 'Portuguese', 
             'Romanian', 'Romansh_Sursilvan', 'Russian', 'Sakha', 'Slovenian', 'Spanish', 
             'Swedish', 'Tamil', 'Tatar', 'Turkish', 'Ukranian', 'Welsh']

with torch.no_grad():
  outputs = model(**input)
print(outputs)
predictions = outputs.logits
prediction = np.argmax(predictions.cpu(), axis=1)
print(languages[prediction])

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1883,  0.9632, -1.6344,  2.0956,  1.0585, -0.5641,  0.5771, -1.1491,
         -0.4986,  1.5071,  1.5290,  1.2281, -1.2052, -0.2889,  0.3573,  0.6550,
          2.4324, -1.0723,  1.2211, -0.2376, -0.6301, -0.3741,  0.0999, 10.0125,
         -2.7314,  0.3732,  1.0742, -0.4474, -1.1650, -0.8459,  0.3956, -0.1116,
         -0.4340, -1.4522, -1.9871,  2.4406, -1.5862, -1.0797, -0.3318, -0.2619,
          0.0648, -0.7279, -0.3586,  2.5571, -0.6700]], device='cuda:0'), hidden_states=None, attentions=None)
Japanese


In [4]:
from langdetect import detect
detect("这是一句中国话。")

'zh-cn'

In [5]:
all_code = set()
for r in results:
  # if r['lang'] == 'te':
  #   print(r)
  #   break
  all_code.add(r['lang'])
print(all_code)

{'fi', 'ko', 'ar', 'ru', 'te', 'bn', 'ja'}


In [ ]:
qid_to_lang, qid_ans_inlanguage, qid_ans_otherlanguage, qid_no_ans = {}, defaultdict(int), defaultdict(int), []
exception_context = []
for r in results:
  # dict_keys(['q_id', 'question', 'answers', 'lang', 'ctxs'])
  qid = r['q_id']
  lang = r['lang']
  qid_to_lang[qid] = lang

  all_no_answer = True
  for c in r["ctxs"]:
    title = c['title']
    context = c['text']
    has_ans = c['has_answer']
    try:
      c_lang = detect(context)
    except:
      exception_context.append(context)
      continue
    if has_ans:
      all_no_answer = False
      if c_lang == lang:
        qid_ans_inlanguage[qid] += 1
      else:
        qid_ans_otherlanguage[qid] += 1

  if all_no_answer:
    qid_no_ans.append(qid)

    # input = tokenizer(context, return_tensors='pt')
    # input = input.to(device)
    # with torch.no_grad():
    #   outputs = model(**input)

    # predictions = outputs.logits
    # prediction = np.argmax(predictions.cpu(), axis=1)
    # c_lang = languages[prediction]

In [ ]:
total = 0
for r in results:
  for c in r["ctxs"]:
    total += 1

total

111980

In [ ]:
len(results)

5599

In [ ]:
len(qid_no_ans)

2889

In [ ]:
qid_no_ans[0]

'-1024677970375145871'

In [ ]:
for r in results:
  if r['q_id'] == "-1024677970375145871":
    for c in r['ctxs']:
      print(c['has_answer'])
    break

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [ ]:
across_lang = set(qid_ans_inlanguage.keys()).intersection(set(qid_ans_otherlanguage.keys()))

In [ ]:
len(across_lang)

146

In [ ]:
to_write = []
for r in results:
  if r['q_id'] in across_lang:
    to_write.append(r)

# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_ans_across_lang.json", "w") as outfile:
#   json.dump(across_lang, outfile)

In [ ]:
with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_ans_across_lang.json") as f:
  across_lang = json.load(f)

In [ ]:
across_lang[0]

{'answers': ['28 జూలై 1914', '1914'],
 'ctxs': [{'ctx_lang': 'te',
   'has_answer': True,
   'id': '30884074',
   'score': '44.05735',
   'text': 'కానీ జర్మన్లు తనేన్బర్గ్ మరియు మస్యూరియన్ లేక్స్ యుద్ధాల్లో తూర్పు ప్రుస్సియా దండయాత్రను ఆపివేశారు. నవంబరు 1914 లో, ఒట్టోమన్ సామ్రాజ్యం సెంట్రల్ పవర్స్లో చేరింది, ఇది కాకసస్ , మెసొపొటేమియా మరియు సీనాయిల్లోప్రారంభ సరిహద్దులను ప్రారంభించింది. 1915 లో, ఇటలీ మిత్రరాజ్యాలు చేరింది మరియు బల్గేరియా సెంట్రల్ పవర్స్లో చేరింది; రోమేనియా 1916 లో మిత్రరాజ్యాలు చేరింది, అలాగే 1917 లో యునైటెడ్ స్టేట్స్ చేసింది. మార్చ్ 1917 లో రష్యన్ ప్రభుత్వం కూలిపోయింది మరియు నవంబరులో ఒక విప్లవం తరువాత మరింత సైనిక ఓటమి కారణంగా రష్యన్లు బ్రెట్స్ట్-లిటోవ్స్క్ ఒప్పందం ద్వారా సెంట్రల్ పవర్స్తో ఒప్పందాలు కుదుర్చుకున్నారు , దీనికి జర్మన్లు \u200b\u200bగణనీయమైన విజయాన్ని అందించారు. 1918 వసంతకాలంలో పశ్చిమ ఫ్రంట్ వెంట ఒక అద్భుతమైన జర్మన్ దాడి తర్వాత, మిత్రరాజ్యాలు విజయవంతంగా పోరాడుతూ , జర్మనీలను విజయవంతమైన దాడుల వరుసలో నడిపించాయి.నవంబరు 4, 1918 న, ఆస్ట్రో-హంగేరియన్',
   'title': 

In [6]:
all_code = {'ar', 'bn', 'ja', 'fi', 'ru', 'ko', 'te'}
for datum in results:
  # src_lang = datum['lang']
  for context in datum['ctxs']:
    if True: #context['has_answer']:
      try:
        tar_lang = detect(context['text'])
        context['ctx_lang'] = tar_lang
      except:
        context['ctx_lang'] = None

In [7]:
# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_ans_results_ctxlang.json", "w") as outfile:
#   json.dump(results, outfile)
# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_results_100_long_ctxlang.json", "w") as outfile:
#   json.dump(results, outfile)

In [ ]:

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel, AutoModelForSeq2SeqLM

MT_models = {}
MT_tokenizers = {}
for code in all_code:
  if code == "te":
    continue
  key = "{}-en".format(code)
  model_path = "Helsinki-NLP/opus-mt-{}-en".format(code)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  MT_models[key] = model
  MT_tokenizers[key] = tokenizer


# ar2en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
# ar2en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

# bn2en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bn-en")
# bn2en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-bn-en")

# ru2en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ja-en")
# ru2en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ja-en")

# ru2en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
# ru2en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

# fi2en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fi-en")
# fi2en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fi-en")

# ko2en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
# ko2en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-en")

In [ ]:
translate = Translation()
translation = translate("This is a test translation into Spanish", "zh")
translation

Downloading:   0%|          | 0.00/916k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/786k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

'这是西班牙文的测试翻译'

In [ ]:
len(results)

3124

In [ ]:
# {'ctx_lang': 'en',
#    'has_answer': True,
#    'id': '17763030',
#    'score': '39.482452',
#    'text': "resulting genocides and the 1918 influenza pandemic caused another 50 to 100 million deaths worldwide. On 28 June 1914, Gavrilo Princip, a Bosnian Serb Yugoslav nationalist, assassinated the Austro-Hungarian heir Archduke Franz Ferdinand in Sarajevo, leading to the July Crisis. In response, on 23 July Austria-Hungary issued an ultimatum to Serbia. Serbia's reply failed to satisfy the Austrians, and the two moved to a war footing. A network of interlocking alliances enlarged the crisis from a bilateral issue in the Balkans to one involving most of Europe. By July 1914, the great powers of Europe were divided into two coalitions:",
#    'title': 'World War I'},
import gc

all_data = []
final_data = []
# for datum in across_lang:
for idx, datum in enumerate(results):
  # if idx <= 500*7:
  #   continue

  new_datum = []
  src_lang = datum['lang']
  qn = datum['question']
  answer = datum['answers']
  qn_en = translate(qn, 'en')
  ans_en = [translate(ans, 'en') for ans in answer]
  all_context_lang = set([context['ctx_lang'] for context in datum['ctxs'] if context['ctx_lang'] is not None])
  if len(all_context_lang) == 1:
    continue
  for context in datum['ctxs']:
    # if context['has_answer']:
      tar_lang = context['ctx_lang']
      if "zh" in tar_lang:
        tar_lang = "zh"
      sample = {}
      if tar_lang == src_lang:
        for k in datum:
          if k != 'ctxs':
            sample[k] = datum[k]
        sample['context'] = context
        sample['orig_lang'] = src_lang
        sample['orig_qn'] = qn
        new_datum.append(sample)
      # elif tar_lang in all_code: #tar_lang != src_lang
      else:
        try:
          translated_qn = translate(qn_en, tar_lang)
          translated_ans = [translate(ans, tar_lang) for ans in ans_en]
        except:
          print("caught an exception, tar_lang is {}".format(tar_lang))
          continue
        for k in datum:
          if k != 'ctxs':
            sample[k] = datum[k]
        sample['question'] = translated_qn
        sample['answers'] = translated_ans
        sample['lang'] = tar_lang
        sample['context'] = context
        sample['orig_lang'] = src_lang
        sample['orig_qn'] = qn
        new_datum.append(sample)
  all_data.append(new_datum)
  final_data.append(new_datum)

  if idx % 50 == 0 and idx != 0:
    with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_part{idx}.json".format(idx=idx//50), "w") as outfile:
      json.dump(all_data, outfile)
      print("Wrote to xor_dev_dpr_retrieval_xlingual_samples_nofilter_part{idx}.json".format(idx=idx/500))
      all_data = []
      torch.cuda.empty_cache()
      gc.collect()

with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_final.json", "w") as outfile:
      json.dump(final_data, outfile)
      print("Wrote to xor_dev_dpr_retrieval_xlingual_samples_nofilter_final.json") 

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/296M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/690k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

caught an exception, tar_lang is es


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/285M [00:00<?, ?B/s]

caught an exception, tar_lang is sv
caught an exception, tar_lang is en
caught an exception, tar_lang is es


KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi #--gpu-reset -i 0

Tue May 10 03:52:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |  15000MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples.json") as f:
  xlingual_samples = json.load(f)

xlingual_samples[-1]

[{'answers': ['29개', '29', '29'],
  'context': {'ctx_lang': 'ko',
   'has_answer': True,
   'id': '26079640',
   'score': '38.419136',
   'text': '나누기도 한다. 고대불어는 라틴어에서 분화한 여러 로망스어 방언 중 북부 프랑스의 오일어 방언, 특히 프랑시앵(francien) 방언을 일컫는 것이며, 중기 프랑스어는 고대와 근대의 과도기적 형태이며, 근대불어는 프랑스사에서 대세기()이라 불리는 17세기에 플레야드, 몰리에르 등의 고전주의 작가들이 라틴어와 이탈리아어의 문법과 어휘를 받아들여 완성한 것이기에 중기 프랑스어와는 확연한 차이가 나게 된다. 17세기 작가 장 드 라퐁텐의 우화집의 원문을 프랑스의 아동들도 읽을 수 있을 정도로 근대불어는 현대에 와서도 거의 변하지 않은 데 비하여 중기불어는 직해하기가 좀 더 어려워지고, 고대 프랑스어는 전문지식이 없으면 불어 원어민도 독해할 수 없다. 프랑스어는 29개 나라에서 공식 언어로 지정되어 있고, 그 중 대부분의 나라가 프랑스어를 사용하는 커뮤니티인 프랑코포니를 구성한다. 또한 국제',
   'title': '프랑스어'},
  'lang': 'ko',
  'orig_lang': 'ko',
  'orig_qn': '프랑스어를 국어로 사용하는 국가는 몇 개국인가?',
  'q_id': '-437495997697322866',
  'question': '프랑스어를 국어로 사용하는 국가는 몇 개국인가?'},
 {'answers': ['Dua puluh sembilan.', '29', '29'],
  'context': {'ctx_lang': 'id',
   'has_answer': True,
   'id': '44639618',
   'score': '37.850197',
   'text': 'Bahasa Perancis ("le français", "la langue française") meru

In [ ]:
len(xlingual_samples)

146

In [ ]:
with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_final.json") as f:
  nofilter_samples = json.load(f)

# nofilter_samples[-7]

In [ ]:
len(nofilter_samples)

3124

In [ ]:
nofilter_samples[0]

[{'answers': ['ఇంగ్లాండ్', 'భారత ఉపఖండం'],
  'context': {'ctx_lang': 'te',
   'has_answer': False,
   'id': '30739502',
   'score': '39.741467',
   'text': 'దేవి అలతారం కోలహలమ్మ కోలాహలమ్మ పేరుతో ఇక్కడ కొల్హాపురం నిర్మిచాడు. కోలహలమ్మ దేవత పూరుతో ఈ నగరం నిర్మించబడిందని ప్రాంతీయ కథనాలు వివరిస్తున్నాయి. చోళపాలకులు వీరరాజేంద్ర చోళుడు (వీరచోళుడు), విక్రమచోళుడు మరియు రాజేంద్రచోళుడు (రాజరాజనరేంద్రచోళుడు) స్థాపించిన శిల్పాలలో మొదటి అవని కోలార్, ముల్బగల్, సిట్టి బెట్టా మరియు ఇతర ప్రాంతాల కొన్ని శిలాక్షరరూప వివరాలు లభిస్తున్నాయి. ఈ శిలాక్షరాలు కోలార్\u200cను " నికరిలి చోళమండలం, జయం కొండ చోళమండలం అని ప్రస్తావిస్తున్నాయి. మొదటి రాజేంద్రచోళుడు కూడా కొలరమ్మ ఆలయం సందర్శించాడు. చోళుల కాలంలో మారికుప్పం గ్రామంలో ఉన్న సోమేశ్వరాలయం, శ్రీ ఉద్దండేశ్వరాలయం, ఉరుగంపేట్\u200cలో ఈశ్వరాలయం, మదివాల గ్రామంలో శివాలయం మొదలైన పలు శివాలయాలు కూడా నిర్మించబడ్డాయి. కోలార్\u200cలో క్రీ.శ 1116 వరకు చోళులపాలన కొనసాగింది. దురదృష్టకరంగా కోలార్ లోని చోళుల శిలాశాసనాలు నిర్లక్ష్యానికి గురైయాయి. కొన్ని సాంస్కృతిక దౌర్జన్యానికి గురయ